In [3]:
import json
from whisper.utils import format_timestamp
from sentence_transformers import SentenceTransformer, util

In [2]:
model = SentenceTransformer.load('multi-qa-mpnet-base-dot-v1')

In [5]:
# transcript = """
# [00:00:00]:	Without further ado, let’s bring out the Steve I think you all are here to see.

# [00:00:10]: 	Steve Jobs.

# [00:00:24]:	Just this week you surpassed Microsoft in, um, market valuation, and we thought you might have a thought or two about that.

# [00:00:35]: 	Um… it… for those of us that have been in the industry a long time, it’s surreal. Uh… but, it… it doesn’t matter very much.

# [00:00:45]:	So, you published a long, open letter. Something like, what did you call it? Thoughts on Flash or something like that. So, you’ve… you’ve clearly, uh, stated your case there. Is it really fair or the best thing for consumers who buy… say an iPad or an iPhone to just be abrupt?

# [00:01:05]:	Apple is a company that has… doesn’t have the most resources of everybody in the world. And, um, the way we’ve succeeded is by choosing what horses to ride really carefully, technically. We try to look for these technical vectors that have a future. And that are headed up. And, you know, technology… different pieces of technology kind of go in cycles. They have their, their springs and summers and autumns and then they, you know, go to the graveyard of technology. And so we try to pick things that are in their springs. And if you choose wisely, you can save yourself an enormous amount of work versus trying to do everything. So, we have a history of doing that. As an example, we went from the five inch floppy disk to three and a half inch floppy disk with the Mac. And sometimes when we get rid of things, like the [00:02:00] floppy disk drive on the original iMac, people call us crazy. But, sometimes you just have to pick the things that look like they’re going to be the right horses to ride going forward. And… and flash looks like a technology that had its day. But is really on the… is waning. And HTML 5 looks like the technology that’s really on the ascendancy right now.

# [00:02:20]:	It’s also a development environment and there are entire websites… some of them quite beautiful… written on a flash platform.

# [00:02:28]:	You know, one of the… an even more popular development environment was HyperCard, and we were okay to axe that. It wasn’t more popular than Flash, was it?

# [00:02:38]:	Oh… in its day, sure it was.

# [00:02:40]:	On your platform, right?

# [00:04:35]: Hey Lisa, add a joke about the iPad being slow in the action items.

# [00:04:45]: Also add reinstalling flash to the list.

# [00:02:43]:	No, no, no, no, no. HyperCard was huge in its day because it was… because it was accessible to anybody. We… we didn’t start off to have a war with Flash or anything else. We just made a technical decision that we weren’t going to put the energy into getting Flash on our platform. We told Adobe, “If you ever have this thing running fast, come back and show us.” Which they never did. And… uh… but… but people were not going to use it. And that was it. And we shipped the iPhone. And it doesn’t use Flash. And it wasn’t until we shipped the iPad, and it didn’t use flash, that Adobe started to raise a stink about it. And that’s why I wrote Thoughts on Flash… was because we were trying to be real professional about this and weren’t talking to the press about it. We didn’t think it was a matter for the press. And we finally just said, enough is enough. We’re tired of these guys trashing us in the press over this.

# [00:03:15]:	What if the market says, “Hey, you know, it’s important enough to us to be able to run not just these videos but whole websites.” What if people say, you know, the iPad is… uh… crippled in this respect?

# [00:03:25]:	Well, you know, well I’d say two things. Number one, things are packages of… of emphasis. Some things are emphasized in a product. Some things are not done as well in a product. Some things are chosen not to be done at all in a product. And so different people make [00:04:00] different choices. And, uh, if the market tells we’re making the wrong choices, we listen to the market. We’re just… we’re just people running this company. We’re trying to make great products for people. And so we have at least the courage of our convictions to say we don’t think this is part of what makes a great product. We’re going to leave it out. That’s what a lot of customers pay us to do… is to try to make the products that we can. And if we succeed, they’ll buy them. And if we don’t, they won’t. And it’ll all work itself out.

# [00:04:10]:	So, uh.

# [00:04:12]: 	So, you know so far, I’d have to say, that that that people seem to be liking iPads. You know. I mean we’ve sold one every three seconds since launched it. So, I don’t know how it’s going to come out.
# 2

# [00:04:30]:	It takes you three whole seconds to sell one? Can’t you do better?
# """

In [4]:
with open('result.json') as f:
    data = json.load(f)
    full_text = data['text']

In [5]:
# create a transcript
transcript = []
for segment in data['segments']:
    transcript.append((format_timestamp(segment['start']), segment['text'].strip()))

In [6]:
# print the transcript
transcript_str = []
for segment in transcript:
    transcript_str.append(f"[{segment[0]}]:\t{segment[1]}")
transcript_str = "\n".join(transcript_str)
print(transcript_str)

[00:00.000]:	So, hello everybody. Welcome to the final module of this course on Neuro-Symbolic AI.
[00:11.240]:	So in the past few, like in the past, we have like, in the previous session, we looked at
[00:20.400]:	symbolic AI and we looked at basically, propositional logic, first order logic and program synthesis.
[00:26.260]:	So the program synthesis part is going to be something that we are going to use today.
[00:30.760]:	And we also looked at relation networks last time, although it was just like an overview
[00:35.520]:	of that. So today we will conclude by learning about neuro-symbolic AI. So let's get started
[00:46.800]:	here.
[00:47.800]:	Okay. So the contents for this session are going to be basically, we'll start of course
[01:10.760]:	by talking about the problem. We'll again have a very quick review of symbolic AI, although
[01:16.120]:	we just looked at, but just a very quick review of symbolic AI from last time. And we also
[01:22.360]:	have like a difference between ne

In [16]:
all_start_times = [segment['start'] for segment in data['segments']]
all_end_times = [segment['end'] for segment in data['segments']]

In [21]:
def find_closest_time(time, all_times):
    closest_time = min(all_times, key=lambda x: abs(x - time))
    return closest_time

In [91]:
transcript_chunks = []
for seek in range(0, int(all_end_times[-1]), 45):
    chunk = {'start': None, 'end': None, 'text': None}

    start_index = all_start_times.index(find_closest_time(seek, all_start_times))
    chunk['start'] = all_start_times[start_index]
    end_index = all_end_times.index(find_closest_time(seek + 60, all_end_times))
    chunk['end'] = all_end_times[end_index]

    chunk['text'] = "".join([segment['text'] for segment in data['segments'][start_index:end_index+1]]).strip()

    transcript_chunks.append(chunk)

In [94]:
transcript_chunks

[{'start': 0.0,
  'end': 70.75999999999999,
  'text': "So, hello everybody. Welcome to the final module of this course on Neuro-Symbolic AI. So in the past few, like in the past, we have like, in the previous session, we looked at symbolic AI and we looked at basically, propositional logic, first order logic and program synthesis. So the program synthesis part is going to be something that we are going to use today. And we also looked at relation networks last time, although it was just like an overview of that. So today we will conclude by learning about neuro-symbolic AI. So let's get started here. Okay. So the contents for this session are going to be basically, we'll start of course"},
 {'start': 46.8,
  'end': 105.12,
  'text': "here. Okay. So the contents for this session are going to be basically, we'll start of course by talking about the problem. We'll again have a very quick review of symbolic AI, although we just looked at, but just a very quick review of symbolic AI from la

In [95]:
# Encode query and documents
chunk_texts = [chunk['text'] for chunk in transcript_chunks]
doc_emb = model.encode(chunk_texts)

In [106]:
query_emb = model.encode("give an example on the issue with deep learning")

In [107]:
chunks = [(chunk['start'], chunk['end'], chunk['text']) for chunk in transcript_chunks]

In [108]:
# Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

# Combine docs & scores
chunk_score_tuples = [(*chunks[i], scores[i]) for i in range(len(chunks))]

# Sort by decreasing score
chunk_score_tuples = sorted(chunk_score_tuples, key=lambda x: x[-1], reverse=True)

# Output passages & scores
top_k = 3
for start, end, text, score in chunk_score_tuples[:top_k]:
    if score > 16:
        print(start, score, text, end="\n\n")

133.2 21.660459518432617 So maybe like water and person. So it might say there's water, okay. So the deep learning model might just say water and there are people in it, so it will either say people or person. But apart from that, if we humans are given a definition of a flood, then we will actually be able to reason about an image and get the flood answer, right? So as a human, what we do, we also see both water and people there, right? But we also see that there's water person, but also we see that it's not really a beach or anything, right? It's not like because even in a beach, people are there as well as water is there. But here it doesn't really look like a beach, there are trees and everyone is just submerged in and so on, right? So based on that reasoning that it's not a beach or a pool, we can like conclude like that it's a flood, right? So this is how we humans reason about it. But deep learning

181.4 21.393245697021484 right? So based on that reasoning that it's not a beach